In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: MacOSX


In [2]:
import requests
import pandas as pd

In [3]:
def get_historic_price(symbol, exchange='bitfinex', after='2018-01-01'):
    url = 'https://api.cryptowat.ch/markets/{exchange}/{symbol}usd/ohlc'.format(
        symbol=symbol, exchange=exchange)
    resp = requests.get(url, params={
        'periods': '3600',
        'after': str(int(pd.Timestamp(after).timestamp()))
    })
    resp.raise_for_status()
    data = resp.json()
    df = pd.DataFrame(data['result']['3600'], columns=[
        'CloseTime', 'OpenPrice', 'HighPrice', 'LowPrice', 'ClosePrice', 'Volume', 'NA'
    ])
    df['CloseTime'] = pd.to_datetime(df['CloseTime'], unit='s')
    df.set_index('CloseTime', inplace=True)
    return df

In [4]:
last_week = (pd.Timestamp.now() - pd.offsets.Day(7))
last_week

Timestamp('2022-08-20 00:21:07.600824')

In [5]:
btc = get_historic_price('btc', 'bitstamp', after=last_week)

In [6]:
eth = get_historic_price('eth', 'bitstamp', after=last_week)

In [7]:
#描述統計，count（非空值数）、unique（唯一值数）、top（频数最高者）、freq（最高频数），计数，平均值，标准差，最小值，最大值
btc.describe()

,OpenPrice,HighPrice,LowPrice,ClosePrice,Volume,NA
count,168.000000,168.000000,168.000000,168.000000,168.000000,1.680000e+02
mean,21351.202381,21433.851190,21269.476190,21347.071429,68.413230,1.455967e+06
std,240.986932,236.983151,259.409416,251.182415,92.050309,1.944292e+06
min,20616.000000,20633.000000,20170.000000,20271.000000,4.985691,1.072799e+05
25%,21227.000000,21316.750000,21141.000000,21231.750000,25.613658,5.471490e+05
50%,21404.500000,21471.500000,21317.000000,21408.000000,41.877971,8.984720e+05
75%,21517.000000,21590.750000,21449.250000,21514.500000,73.340413,1.519155e+06
max,21750.000000,21892.000000,21682.000000,21746.000000,749.508166,1.562005e+07


In [8]:
btc.head()

,OpenPrice,HighPrice,LowPrice,ClosePrice,Volume,NA
CloseTime,,,,,,
2022-08-20 01:00:00,20840,21186,20811,21011,309.508657,6.480234e+06
2022-08-20 02:00:00,21007,21102,20990,21000,29.459364,6.199347e+05
2022-08-20 03:00:00,21011,21183,20996,21143,46.398594,9.801168e+05
2022-08-20 04:00:00,21140,21176,21100,21149,17.827880,3.767775e+05
2022-08-20 05:00:00,21150,21297,21086,21264,49.233074,1.042918e+06


In [9]:
eth['ClosePrice'].plot(figsize=(15, 7))

<AxesSubplot:xlabel='CloseTime'>

In [10]:
eth.head()

,OpenPrice,HighPrice,LowPrice,ClosePrice,Volume,NA
CloseTime,,,,,,
2022-08-20 01:00:00,1608.6,1638.0,1608.6,1625.2,495.753116,805448.639221
2022-08-20 02:00:00,1626.3,1633.0,1615.5,1619.0,315.669042,512140.464268
2022-08-20 03:00:00,1619.0,1636.4,1617.3,1628.4,179.889453,292880.944797
2022-08-20 04:00:00,1629.9,1634.0,1623.0,1629.4,229.656851,373825.059171
2022-08-20 05:00:00,1627.8,1653.4,1626.2,1650.4,371.816458,607725.565457


In [11]:
#動態圖表
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook

In [12]:
output_notebook()

Loading BokehJS ...

In [13]:
p1 = figure(x_axis_type="datetime", title="Crypto Prices", width=800)
p1.grid.grid_line_alpha=0.3
p1.xaxis.axis_label = 'Date'
p1.yaxis.axis_label = 'Price'

p1.line(btc.index, btc['ClosePrice'], color='#f2a900', legend_label='Bitcoin')
p1.line(eth.index, eth['ClosePrice'], color='#A6CEE3', legend='Ether')

p1.legend.location = "top_left"

show(p1)